In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import geopandas as gpd
from geoalchemy2 import Geometry
import shapely.wkb as wkb


In [2]:
# Leer el archivo CSV
df_incendios = pd.read_csv(r'G:\Mi unidad\OSPA\01. Tematicas\04. Incendios\01. Productos\postmodelo_icv\temporales\probabilidad_icv.csv', sep=';', encoding='utf-8')
df_deslizamientos = pd.read_csv(r'G:\Mi unidad\OSPA\01. Tematicas\03. Deslizamientos\01. Productos\postmodelo_idd\temporales\probabilidad_idd.csv', sep=';', encoding='utf-8')


In [3]:
# Crear una conexión a la base de datos
usuario = 'postgres'
contraseña = 'Septiembre0672'
host = 'localhost'
puerto = '5432'
db = 'alertas'
url_conexion = f'postgresql://{usuario}:{contraseña}@{host}:{puerto}/{db}'

In [4]:
def alertas_incendios(url, df_incendios):
    try:
        engine = create_engine(url_conexion)
        return df_incendios.to_sql('incendios', con=engine, if_exists='replace', index=False)
    except:
        engine = create_engine(url_conexion)
        # Eliminar los registros existentes de la tabla 'incendios'
        with engine.connect() as conn:
            conn.execute("DELETE FROM incendios")
        return df_incendios.to_sql('incendios', con=engine, if_exists='append', index=False)

In [5]:
alertas_incendios(url_conexion, df_incendios)

59

In [6]:
def alertas_deslizamientos(url, df_deslizamientos):
    try:
        engine = create_engine(url_conexion)
        return df_deslizamientos.to_sql('deslizamientos', con=engine, if_exists='replace', index=False)
    except:
        engine = create_engine(url_conexion)
        # Eliminar los registros existentes de la tabla 'incendios'
        with engine.connect() as conn:
            conn.execute("DELETE FROM deslizamientos")
        return df_deslizamientos.to_sql('deslizamientos', con=engine, if_exists='append', index=False)

In [7]:
alertas_deslizamientos(url_conexion, df_deslizamientos)

81

In [8]:
def vista_departamentos(url_conexion):
    
    engine = create_engine(url_conexion)
    
    create_view_sql = """
CREATE OR REPLACE VIEW departamentos_colombia AS
SELECT departamen, jsonb_build_object(
  'type',       'Feature',
  'geometry',   ST_AsGeoJSON(geom, 4)::jsonb,
  'properties', jsonb_build_object(
    'DEPARTAMENTO', departamen
  )
) AS geom
FROM public.departamentos_wgs84

"""
    try:
        with engine.connect() as conn:
            conn.execute(create_view_sql)
            return("Vista creada exitosamente.")
    except SQLAlchemyError as e:
        return(f"Error al crear la vista: {e}")

In [9]:
vista_departamentos(url_conexion)

'Vista creada exitosamente.'

In [10]:
def vista_deslizamientos(url_conexion):
    
    engine = create_engine(url_conexion)
    
    create_view_sql = """
CREATE OR REPLACE VIEW alertas_deslizamientos AS
SELECT d.*,
jsonb_build_object(
  'type', 'Feature',
  'geometry', ST_AsGeoJSON(ST_GeometryN(mg.geom, 1), 4)::jsonb,
  'properties', jsonb_build_object(
    'PROBABILIDAD_DESC', CASE
                           WHEN d."PROBABILIDAD" = 1 THEN 'BAJA'
                           WHEN d."PROBABILIDAD" = 2 THEN 'MEDIA'
                           WHEN d."PROBABILIDAD" = 3 THEN 'ALTA'
                           ELSE 'no definida' -- Opcional: para valores distintos de 1, 2 o 3
                         END,
    'PROBABILIDAD', d."PROBABILIDAD",
    'REGION', d."REGION",
    'DEPARTAMENTO', d."DEPARTAMENTO",
    'MUNICIPIO', d."MUNICIPIO",
	  'FECHA_EJECUCION', d."FECHA_EJECUCION"
  )
) AS geom
FROM deslizamientos AS d
INNER JOIN mgn_mpio_politico AS mg
ON d."COD_DANE" = mg.mpio_cdpmp;

"""
    try:
        with engine.connect() as conn:
            conn.execute(create_view_sql)
            return("Vista creada exitosamente.")
    except SQLAlchemyError as e:
        return(f"Error al crear la vista: {e}")

In [11]:
vista_deslizamientos(url_conexion)

'Vista creada exitosamente.'

In [12]:
def vista_incendios(url_conexion):
    
    engine = create_engine(url_conexion)
    
    create_view_sql = """
CREATE OR REPLACE VIEW alertas_incendios AS
SELECT i.*, jsonb_build_object(
  'type',       'Feature',
  'geometry',   ST_AsGeoJSON(ST_GeometryN(mg.geom, 1), 4)::jsonb,
  'properties', jsonb_build_object(
	 'PROBABILIDAD_DESC', CASE
                           WHEN i."PROBABILIDAD" = 1 THEN 'BAJA'
                           WHEN i."PROBABILIDAD" = 2 THEN 'MEDIA'
                           WHEN i."PROBABILIDAD" = 3 THEN 'ALTA'
                           ELSE 'no definida' -- Opcional: para valores distintos de 1, 2 o 3
                         END,
    'PROBABILIDAD', i."PROBABILIDAD",
    'REGION', i."REGION",
    'DEPARTAMENTO', i."DEPARTAMENTO",
	  'FECHA_EJECUCION', i."FECHA_EJECUCION",
    'MUNICIPIO', i."MUNICIPIO"
  )
) AS geom
FROM incendios as i
INNER JOIN mgn_mpio_politico as mg
ON i."COD_DANE" = mg.mpio_cdpmp;

"""
    try:
        with engine.connect() as conn:
            conn.execute(create_view_sql)
            return("Vista creada exitosamente.")
    except SQLAlchemyError as e:
        return(f"Error al crear la vista: {e}")

In [13]:
vista_incendios(url_conexion)

'Vista creada exitosamente.'

In [14]:
# Carga tu archivo SHP
gdf = gpd.read_file(r'C:\Users\Ever Salazar\Documents\VisorAlertas\server\src\Departamentos_WGS84\Departamentos_WGS84.shp')

# Configura la conexión a tu base de datos
engine = create_engine(url_conexion)


In [19]:
gdf = gdf.rename(columns=str.lower)


In [20]:
gdf.columns

Index(['acto_admin', 'observacio', 'departamen', 'cod_depart', 'shape_leng',
       'shape_area', 'areaha', 'geometry'],
      dtype='object')

In [21]:
# Convertir la columna de geometría a WKB
gdf['geometry'] = gdf['geometry'].apply(lambda x: wkb.dumps(x, hex=True))

C:\Users\Ever Salazar\AppData\Local\Temp\ipykernel_32220\3342600575.py:2: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].apply(lambda x: wkb.dumps(x, hex=True))


In [22]:
gdf

,acto_admin,observacio,departamen,cod_depart,shape_leng,shape_area,areaha,geometry
0,None,None,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA,88,0.652403,0.004074,0.0,0106000000030000000103000000010000005E04000050...
1,None,None,CAUCA,19,13.769394,2.534944,0.0,01060000000300000001030000000100000051000000C0...
2,None,None,ATLÁNTICO,08,2.529379,0.274446,0.0,01030000000100000073090000FC42989ED8B552C01E22...
3,None,None,RISARALDA,66,4.661057,0.290082,0.0,010300000001000000412200002830E03C870053C05582...
4,None,None,ANTIOQUIA,05,21.109882,5.147846,0.0,0103000000010000005E4F0000B48F7A27BD1953C0362A...
...,...,...,...,...,...,...,...,...
88,None,None,VALLE DEL CAUCA,76,12.222911,1.678614,0.0,01030000000100000032000000F8420C40534F53C06AF1...
89,None,None,VALLE DEL CAUCA,76,12.222911,1.678614,0.0,01030000000100000027000000CC4A0A80965B53C0D729...
90,None,None,VALLE DEL CAUCA,76,12.222911,1.678614,0.0,0103000000010000000C0000005C41F33FF35D53C09D2B...
91,None,None,VALLE DEL CAUCA,76,12.222911,1.678614,0.0,01030000000100000025000000BC900480835F53C06C2F...


In [ ]:
gdf.to_sql('intento', engine, if_exists='replace', index=False, 
           dtype={'geometry': Geometry(geometry_type='GEOMETRY', srid=4326)})